**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Feb 12 00:21:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 01.SOLUTIONS-TO-CONCEPTS.pdf
 10.pdf
'12 documents.pdf'
 12.pdf
 19106085.pdf
'19106085 shiv shambu bhagat applied mathematics-1.pdf'
'19106085 shiv shambu bhagat CP.pdf'
'19106085 -shiv shambu bhagat-CSCI-101-CP-SET-A.pdf'
'19106085 shiv shambu bhagat EME.pdf'
'19106085 Z-TRANSFORM.pdf'
 22_op_amps1.pdf
 881033108947821_signed.pdf
 881033111280254_signed.pdf
 aadh43.pdf
 aadha4.pdf
 AA.pdf
 AdmitCard_janury.pdf
'AdmitCard_jee mains.pdf'
 Age_Declaration_Form.pdf
 B612_20180409_104610.jpg
 B612_20180409_164200.jpg
 B612_20180409_165456.jpg
 B612_20180409_165642.jpg
 B612_20180409_165753.jpg
 B612_20180409_165908.jpg
 B612_20180409_170030.jpg
 BeautyPlusMe_20170614203944_save.jpg
 BeautyPlusMe_20170615060807_fast.jpg
 BeautyPlusMe_20170615083907_fast.jpg
 BeautyPlusMe_20170615083931_fast.jpg
 BeautyPlusMe_20170615180427_fast.jpg
 BeautyPlusMe_20170619151425_fast.jpg
 BeautyPlusMe_20170626161954_fast.jpg
 BeautyPlusMe_20170626162039_fast.jpg
 BeautyPlusMe_2017

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14691, done.
remote: Total 14691 (delta 0), reused 0 (delta 0), pack-reused 14691
Receiving objects: 100% (14691/14691), 13.25 MiB | 22.32 MiB/s, done.
Resolving deltas: 100% (10000/10000), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-02-12 00:24:07--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  96.4MB/s    in 1.6s    

2021-02-12 00:24:09 (96.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/image.zip -d data/obj

Archive:  /mydrive/yolov3/image.zip
  inflating: data/obj/21Z2Bbj0RdqL._SX342_.txt  
  inflating: data/obj/26a17d71-91f4-4773-8697-1bac3a67b7ed.__CR00970600_PT0_SX970_V1___.jpg  
  inflating: data/obj/26a17d71-91f4-4773-8697-1bac3a67b7ed.__CR00970600_PT0_SX970_V1___.txt  
  inflating: data/obj/33tfc004-36r4p-02_06_1400x.jpg  
  inflating: data/obj/33tfc004-36r4p-02_06_1400x.txt  
  inflating: data/obj/33tfc004-36r4p-02_07.jpg  
  inflating: data/obj/33tfc004-36r4p-02_07.txt  
  inflating: data/obj/51O2BWiir50L._SL1100_.jpg  
  inflating: data/obj/51O2BWiir50L._SL1100_.txt  
  inflating: data/obj/75cmPEtrafficconepvctrafficconeledlighttrafficcone30trafficconeroadsafetycones_2d467820-c1a5-4db0-9455-4980d54821c7Jpg.jpg  
  inflating: data/obj/75cmPEtrafficconepvctrafficconeledlighttrafficcone30trafficconeroadsafetycones_2d467820-c1a5-4db0-9455-4980d54821c7Jpg.txt  
  inflating: data/obj/100cm-5kgs-Red-Color-Black-Base-Plastic-PE-Road-Traffic-Cone-Safety-Cone-CC-A04-.jpg  
  inflating: dat

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.847170 0.589474 0.109434 0.252632

['15', '0.847170', '0.589474', '0.109434', '0.252632']
0 data/obj/images227.txt
0 0.847170 0.589474 0.109434 0.252632
15 0.505660 0.515789 0.098113 0.189474

['15', '0.505660', '0.515789', '0.098113', '0.189474']
0 data/obj/images227.txt
0 0.505660 0.515789 0.098113 0.189474
15 0.303774 0.471053 0.064151 0.163158

['15', '0.303774', '0.471053', '0.064151', '0.163158']
0 data/obj/images227.txt
0 0.303774 0.471053 0.064151 0.163158
15 0.160377 0.439474 0.041509 0.142105

['15', '0.160377', '0.439474', '0.041509', '0.142105']
0 data/obj/images227.txt
0 0.160377 0.439474 0.041509 0.142105
15 0.054717 0.423684 0.041509 0.110526

['15', '0.054717', '0.423684', '0.041509', '0.110526']
0 data/obj/images227.txt
0 0.054717 0.423684 0.041509 0.110526
15 0.418868 0.421053 0.037736 0.105263

['15', '0.418868', '0.421053', '0.037736', '0.105263']
0 data/obj/images227.txt
0 0.418868 0.421053 0.037736 0.105263
15 0.500000 0.510000 0.980000 0.960000

['15', '0.50

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/images222.jpg', 'data/obj/51O2BWiir50L._SL1100_.jpg', 'data/obj/images137.jpg', 'data/obj/images295.jpg', 'data/obj/images172.jpg', 'data/obj/images327.jpg', 'data/obj/images353.jpg', 'data/obj/images31.jpg', 'data/obj/images311.jpg', 'data/obj/images183.jpg', 'data/obj/images207.jpg', 'data/obj/images297.jpg', 'data/obj/images131.jpg', 'data/obj/images233.jpg', 'data/obj/images127.jpg', 'data/obj/images123.jpg', 'data/obj/images313.jpg', 'data/obj/images399.jpg', 'data/obj/images345.jpg', 'data/obj/images224.jpg', 'data/obj/images.jpg', 'data/obj/images348.jpg', 'data/obj/images223.jpg', 'data/obj/images21.jpg', 'data/obj/images170.jpg', 'data/obj/egMiniMetalCNCcuttingmachineCNCWoodRouterVCT4030C_2a0fb2b5-bd99-47ae-94fc-3b50e3778085.jpg', 'data/obj/images176.jpg', 'data/obj/images186.jpg', 'data/obj/images418.jpg', 'data/obj/images361.jpg', 'data/obj/images285.jpg', 'data/obj/images215.jpg', 'data/obj/images191.jpg', 'data/obj/images36.jpg', 'data/obj/images151.jpg', 'data/

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [1]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

/bin/bash: ./darknet: No such file or directory
